<a href="https://colab.research.google.com/github/jlosar/Julio-Losa-ML-projeto/blob/main/MVP_Machine_Learning_%26_Analytics_Julio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - Machine Learning & Analytics


*   Julio Cesar Losa Rodrigues
*   Matricula: 4052025001006
*   Dataset utilizado: Rossmann Store Sales (Kaggle)


# 1. Introdução:



*   **Objetivo:**

Fazer previsão de vendas futuras por loja e produto utilizando técnicas de Machine Learning.

*   **Descrição do problema**

Para qualquer gestor em uma empresa que trabalha com varejo fazer uma boa previsão do volume de vendas é condição fundamental para o planejamento do ano em relação a previsão de receita, Logística e investimento (verba para promoções, Mão de obra e etc.). Esse trabalho tem por objetivo tornar a previsão de vendas mais precisa, confiável e a força de vendas mais eficiênte.

*   **Descrição do Dataset:**

O dataset Rossmann Store Sales foi disponibilizado pelo Kaggle em uma competição de previsão de vendas. Ele contém informações de vendas diárias de 1.115 lojas da rede de farmácias Rossmann na Europa, durante o período de janeiro/2013 a julho/2015 (~2 anos e 7 meses).

*   **Desafios do Dataset:**

      1.   Alta variabilidade: vendas variam por loja, tipo, promoções e sazonalidade.
      2.   Dados faltantes: algumas lojas não possuem histórico completo de concorrência e promoções.
      3.   Fatores externos: datas especiais, feriados e promoções impactam fortemente as vendas.
      4.   Séries temporais múltiplas: mais de mil lojas diferentes, cada uma com comportamento próprio.








# 2. Carga e Preparação dos dados:

(treino, validação, teste; validação cruzada; sempre evitando vazamento de dados)

## Imports das bibliotecas

In [1]:
# Configuração para não exibir os warnings

import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd #Importar biblioteca para manipulação e análise de dados tabulares.
import numpy as np #Importar biblioteca que lida com operações numéricas e arrays multidimensionais de forma eficiente.
import matplotlib.pyplot as plt #Importar biblioteca usada para criar visualizações estáticas (gráficos, plots e etc.)
import time #Biblioteca "time" para medir o tempo de execução de blocos de código.


from sklearn.model_selection import train_test_split, TimeSeriesSplit #Importa métodos para divisão de dados: aleatória (treino/teste) e sequencial (séries temporais).
from sklearn.compose import ColumnTransformer #Permite aplicar diferentes transformações a colunas específicas do dataset. Ajuda com pipelines de pré-processamento.
from sklearn.preprocessing import OneHotEncoder, StandardScaler # Importa ferramentas para transformar categorias em código binário e padronizar dados numéricos.
from sklearn.pipeline import Pipeline #Cria pipelines de machine learning
from sklearn.dummy import DummyClassifier, DummyRegressor #Importa modelos "dummy" (ou de referência). Eles servem como baseline para comparar o desempenho dos seus modelos reais.
from sklearn.linear_model import LogisticRegression, Ridge #Importa modelos de regressão linear.
      #LogisticRegression é para problemas de classificação
      #Ridge é uma forma de regressão linear para problemas de regressão.

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #Importa os modelos de Random Forest,
from sklearn.cluster import KMeans #Importa um algoritmo de clusterização
from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score, confusion_matrix,
                             mean_absolute_error, mean_squared_error, r2_score,
                             silhouette_score) #Importa diversas métricas de avaliação
from sklearn.model_selection import StratifiedKFold, KFold, TimeSeriesSplit, RandomizedSearchCV #Importa mais classes de validação cruzada.
from scipy.stats import randint, uniform  #Importa distribuições de probabilidade para serem usadas no RandomizedSearchCV.
      #"randint" para parâmetros inteiros.
      #"uniform" para parâmetros de ponto flutuante.

from IPython.display import display #Formatação de tabelas
pd.set_option('display.width', None)        # Remove quebra de linhas
pd.set_option('display.max_columns', None)  # Mostra todas as colunas
pd.set_option('display.max_colwidth', None) # Mostra conteúdo completo
pd.set_option('display.float_format', '{:.2f}'.format) # Duas casas decimais

## Carregando Datasets do GitHub

Inicialmente essa parte foi a mais dificultosa, pois estava baixando o dataset diretamente do kaggle sempre que iria rodar o arquivo. Bem instruído, consegui aprender a baixar para o Github.

In [2]:
# Clona meu repositório do GitHub
!git clone https://github.com/jlosar/Julio-Losa-ML-projeto.git

# Lendo os arquivos CSV direto do repositório no GitHub
train = pd.read_csv("Julio-Losa-ML-projeto/Dados/train.csv")
test = pd.read_csv("Julio-Losa-ML-projeto/Dados/test.csv")
store = pd.read_csv("Julio-Losa-ML-projeto/Dados/store.csv")

# Verificação inicial
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("Store shape:", store.shape)

display(train.head())
display(test.head())
display(store.head())

Cloning into 'Julio-Losa-ML-projeto'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 32 (delta 8), reused 5 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 230.88 KiB | 1.91 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Encountered 2 file(s) that should have been pointers, but weren't:
	Dados/store.csv
	Dados/test.csv
Train shape: (1017209, 9)
Test shape: (41088, 8)
Store shape: (1115, 10)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.00,1,0,0
1,2,3,4,2015-09-17,1.00,1,0,0
2,3,7,4,2015-09-17,1.00,1,0,0
3,4,8,4,2015-09-17,1.00,1,0,0
4,5,9,4,2015-09-17,1.00,1,0,0


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN
1,2,a,a,570.00,11.00,2007.00,1,13.00,2010.00,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.00,12.00,2006.00,1,14.00,2011.00,"Jan,Apr,Jul,Oct"
3,4,c,c,620.00,9.00,2009.00,0,NaN,NaN,NaN
4,5,a,a,29910.00,4.00,2015.00,0,NaN,NaN,NaN


## Carregando os Datasets

## Descrição do dataset


In [3]:
#Informações sobre as vendas, o comportamento dos clientes e a frequência de promoções.

pd.set_option('display.float_format', lambda x: '%.2f' % x) #Comando para configurar o comportamento de exibição do Pandas.
store.describe() #função para identificar a distribuição dos dados, a presença de outliers ou valores discrepantes.

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00,1112.00,761.00,761.00,1115.00,571.00,571.00
mean,558.00,5404.90,7.22,2008.67,0.51,23.60,2011.76
std,322.02,7663.17,3.21,6.20,0.50,14.14,1.67
min,1.00,20.00,1.00,1900.00,0.00,1.00,2009.00
25%,279.50,717.50,4.00,2006.00,0.00,13.00,2011.00
50%,558.00,2325.00,8.00,2010.00,1.00,22.00,2012.00
75%,836.50,6882.50,10.00,2013.00,1.00,37.00,2013.00
max,1115.00,75860.00,12.00,2015.00,1.00,50.00,2015.00


In [4]:
store.isnull().sum()
    #".isnull()": É uma função que percorre cada célula do DataFrame e retorna True se a célula estiver vazia (nula) e False se contiver um valor.
    #".sum()": Soma o resultado de ".isnull()". Como Python trata True como 1 e False como 0, o .sum() simplesmente conta quantos valores nulos existem em cada coluna

,0
Store,0
StoreType,0
Assortment,0
CompetitionDistance,3
CompetitionOpenSinceMonth,354
CompetitionOpenSinceYear,354
Promo2,0
Promo2SinceWeek,544
Promo2SinceYear,544
PromoInterval,544


## Tratando a coluna "CompetitionDistance"

Irei útilizar a Média para preencher os dados das colunas vazias e com isso evitarei que o modelo superestime o impacto da concorrência para lojas sem dados  

In [5]:
# 1. Calcular a mediana da coluna
median_distance = store['CompetitionDistance'].median()

# 2. Preencher os valores ausentes com a mediana
store['CompetitionDistance'].fillna(median_distance, inplace=True)

# 3. Verificar o resultado para garantir que não há mais nulos
print(store['CompetitionDistance'].isnull().sum())

0


## Tratando as Colunas de Competição e de Promoção

O objetivo é transformá-las em uma única variável que capture a idade da concorrência.

In [6]:
# Seu código assume que df_store e df_train foram carregados assim:
# store = pd.read_csv('rossmann/store.csv')
# train = pd.read_csv('rossmann/train.csv')

# Preenchendo os valores nulos com valores seguros para cálculo de data.
store['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
store['CompetitionOpenSinceYear'].fillna(1900, inplace=True)

# Garante que as colunas sejam do tipo inteiro.
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].astype(int)
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].astype(int)

# Preenche valores nulos com 0 ou 'NoPromo', indicando a ausência da promoção 2.
store['Promo2SinceWeek'].fillna(0, inplace=True)
store['Promo2SinceYear'].fillna(0, inplace=True)
store['PromoInterval'].fillna('NoPromo', inplace=True)

# Converte as colunas para o tipo inteiro.
store['Promo2SinceWeek'] = store['Promo2SinceWeek'].astype(int)
store['Promo2SinceYear'] = store['Promo2SinceYear'].astype(int)

# Combina os dataframes de treino e loja em um único dataframe.
# Corrigido: Usando os nomes das suas variáveis
df_merged = pd.merge(train, store, on='Store', how='inner')

# Garante que a coluna de data esteja no formato correto.
df_merged['Date'] = pd.to_datetime(df_merged['Date'])

# Cria a coluna de data de início da concorrência.
df_merged['CompetitionOpenSince'] = pd.to_datetime(
    df_merged['CompetitionOpenSinceYear'].astype(str) + '-' +
    df_merged['CompetitionOpenSinceMonth'].astype(str) + '-01',
    errors='coerce'
)

# Cria a coluna de data de início da Promoção 2.
df_merged['Promo2Start'] = df_merged.apply(
    lambda row: pd.to_datetime(
        f"{row['Promo2SinceYear']}-W{row['Promo2SinceWeek']}-1",
        format='%Y-W%W-%w', errors='coerce'
    ) if row['Promo2SinceYear'] > 0 else pd.NaT, axis=1
)

# Calcula a idade da concorrência em meses.
df_merged['CompetitionAgeInMonths'] = (
    (df_merged['Date'].dt.year - df_merged['CompetitionOpenSince'].dt.year) * 12 +
    (df_merged['Date'].dt.month - df_merged['CompetitionOpenSince'].dt.month)
)

# Calcula a idade da Promoção 2 em meses.
df_merged['Promo2AgeInMonths'] = (
    (df_merged['Date'].dt.year - df_merged['Promo2Start'].dt.year) * 12 +
    (df_merged['Date'].dt.month - df_merged['Promo2Start'].dt.month)
)

# Preenche valores inválidos ou negativos com 0 ou um valor seguro.
df_merged.loc[df_merged['CompetitionAgeInMonths'] < 0, 'CompetitionAgeInMonths'] = 0
df_merged['CompetitionAgeInMonths'].fillna(999, inplace=True)

df_merged.loc[df_merged['Promo2AgeInMonths'] < 0, 'Promo2AgeInMonths'] = 0
df_merged['Promo2AgeInMonths'].fillna(999, inplace=True)

# Remove as colunas originais que não são mais necessárias.
df_merged.drop(
    columns=[
        'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'CompetitionOpenSince',
        'Promo2SinceWeek', 'Promo2SinceYear', 'Promo2Start'
    ],
    inplace=True
)

In [9]:
display(df_merged.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,CompetitionAgeInMonths,Promo2AgeInMonths
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.00,0,NoPromo,82,999.00
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.00,1,"Jan,Apr,Jul,Oct",92,64.00
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.00,1,"Jan,Apr,Jul,Oct",103,51.00
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.00,0,NoPromo,70,999.00
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.00,0,NoPromo,3,999.00


In [10]:
df_merged.isnull().sum()

,0
Store,0
DayOfWeek,0
Date,0
Sales,0
Customers,0
Open,0
Promo,0
StateHoliday,0
SchoolHoliday,0
StoreType,0


# 3. ⁠Divisão dos dados:

Limpeza, transformação, seleção/extração de atributos, engenharia de atributos, etc.

# 4. ⁠Modelagem:

treinar e comparar abordagens e modelos diferentes; quando possível, usar pipelines reproduzíveis.

# 5. ⁠Otimização de hiperparâmetros:

Explorar ajustes relevantes para cada modelo.

# 6. Avaliação:

Utilizar métricas adequadas ao tipo de problema, comparar modelos e discutir limitações e melhorias.

# 7. Boas práticas:

Estabelecer baseline, fixar seeds para reprodutibilidade, relatar recursos computacionais usados e tempo de treino, e documentar as decisões de projeto.